## Installing Libraries Needed

In [1]:
!python -m pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m pip install openpyxl



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Dataset Inventory Loader

In [3]:
import os
import re

base_path = r"G:\My Drive\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018_METADATA.xlsx (metadata)
    JANUARY_2018.CSV (survey)
  April:
    APRIL_2018.CSV (survey)
    APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018.CSV (survey)
    OCTOBER_2018_METADATA.xlsx (metadata)
  Unmatched:
    clean_jan_2018_metadata.csv (survey)
    clean_jan_2018_metadata_sheet2.csv (survey)

Year 2019:
  January:
    JANUARY_2019.CSV (survey)
    JANUARY_2019_METADATA.xlsx (metadata)
  April:
    APRIL_2019.CSV (survey)
    APRIL_2019_METADATA.xlsx (metadata)
  July:
    JULY_2019.CSV (survey)
    JULY_2019_METADATA.xlsx (metadata)
  October:
    OCTOBER_2019.CSV (survey)
    OCTOBER_2019_METADATA.xlsx (metadata)

Year 2022:
  January:
    JANUARY_2022.csv (survey)
    JANUARY_2022_METADATA.xlsx (metadata)
  February:
    FEBRUARY_2022.csv (survey)
   

## Load Dataset Function

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

## Metadata Sheet 1 Function 

In [5]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [6]:
import pandas as pd

# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head()


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES


## Metadata Sheet 2 Function

In [7]:
# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 1)

January_metadata.head()

,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


In [8]:
import pandas as pd

def reshape_sheet2(df):
    reshaped = []
    
    # 1. Clean the dataframe: convert everything to string to avoid format errors
    df = df.fillna('').astype(str)

    current_var = None
    current_desc = None

    # Get initial Variable/Description from headers
    # (Check if headers are real text or just "Unnamed")
    header_var = df.columns[0] if 'Unnamed' not in str(df.columns[0]) else ''
    header_desc = df.columns[1] if 'Unnamed' not in str(df.columns[1]) else ''
    
    if header_var: current_var = header_var.strip()
    if header_desc: current_desc = header_desc.strip()

    for idx, row in df.iterrows():
        # --- COLUMN MAPPING (0-based index) ---
        # col 0: Variable
        # col 1: Description
        # col 2: Label
        # col 3: Start Value (Min)
        # col 4: End Value (Max)
        # col 5: Additional Value
        
        var_col  = row.iloc[0].strip()
        desc_col = row.iloc[1].strip()
        label    = row.iloc[2].strip()
        raw_min  = row.iloc[3].strip()
        raw_max  = row.iloc[4].strip()
        add_val  = row.iloc[5].strip()

        # Forward Fill Logic for Var/Desc
        if var_col: current_var = var_col
        if desc_col: current_desc = desc_col

        # Skip empty rows (if no label, it's likely blank space)
        if not label:
            continue

        # --- THE FIX: STRICT READING ---
        
        # Helper: clean Excel floats (e.g., "1.0" -> "1")
        def clean_num(val):
            if not val: return None
            try:
                # Convert to float first to handle "1.0", then int
                return int(float(val))
            except ValueError:
                # If it's text (like "See Note"), return as is
                return val

        # 1. Read Min
        min_value = clean_num(raw_min)

        # 2. Read Max
        # Logic: If Max column is empty, Max is same as Min. 
        # If Max column has value, use it.
        val_max_cleaned = clean_num(raw_max)
        
        if val_max_cleaned is not None:
            max_value = val_max_cleaned
        else:
            max_value = min_value

        reshaped.append({
            "Variable": current_var,
            "Description": current_desc,
            "Label": label,
            "min_value": min_value,
            "max_value": max_value,
            "additional_value": add_val
        })

    return pd.DataFrame(reshaped)

In [9]:

sheet2_aug2024 = load_dataset(year="2024", month="August", filetype="metadata", sheet_number=1)
sheet2_aug2024_reshaped = reshape_sheet2(sheet2_aug2024)

# Preview the first 40 rows
print("January 2024 (Sheet 2) preview:")
sheet2_aug2024_reshaped.head(40)

January 2024 (Sheet 2) preview:


,Variable,Description,Label,min_value,max_value,additional_value
0,PUFSVYMO_VS1,Survey Month,January,1.0,1.0,
1,PUFSVYMO_VS1,Survey Month,February,2.0,2.0,
2,PUFSVYMO_VS1,Survey Month,March,3.0,3.0,
3,PUFSVYMO_VS1,Survey Month,April,4.0,4.0,
4,PUFSVYMO_VS1,Survey Month,May,5.0,5.0,
5,PUFSVYMO_VS1,Survey Month,June,6.0,6.0,
6,PUFSVYMO_VS1,Survey Month,July,7.0,7.0,
7,PUFSVYMO_VS1,Survey Month,August,8.0,8.0,
8,PUFSVYMO_VS1,Survey Month,September,9.0,9.0,
9,PUFSVYMO_VS1,Survey Month,October,10.0,10.0,


In [17]:
# (Assuming load_dataset is already defined in your environment)
sheet2_aug2024 = load_dataset(year="2024", month="August", filetype="metadata", sheet_number=1)

# Process the data
sheet2_aug2024_reshaped = reshape_sheet2(sheet2_aug2024)

# Define the specific path
output_folder = r"C:\Users\juanp\Documents\Random Uni Acts"
filename = "sheet2_aug2024_reshaped.csv"

# Create folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save
full_path = os.path.join(output_folder, filename)
sheet2_aug2024_reshaped.to_csv(full_path, index=False)

print(f"✅ File successfully saved to: {full_path}")
print(sheet2_aug2024_reshaped.head())

✅ File successfully saved to: C:\Users\juanp\Documents\Random Uni Acts\sheet2_aug2024_reshaped.csv
       Variable   Description     Label  min_value  max_value additional_value
0  PUFSVYMO_VS1  Survey Month   January        1.0        1.0                 
1  PUFSVYMO_VS1  Survey Month  February        2.0        2.0                 
2  PUFSVYMO_VS1  Survey Month     March        3.0        3.0                 
3  PUFSVYMO_VS1  Survey Month     April        4.0        4.0                 
4  PUFSVYMO_VS1  Survey Month       May        5.0        5.0                 
